In [1]:
import datetime
import os
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from notebook_shared import utils

from statsmodels.tsa.seasonal import seasonal_decompose
from pandas import Series

import seaborn as sns
sns.set()
matplotlib.style.use('seaborn-colorblind')

INPUT_FILE = "dataset"
FSIZE = "full" # other: tiny, small

df = pd.read_parquet(utils.get_dataset_path(INPUT_FILE, FSIZE))
mdf = pd.read_parquet(utils.get_dataset_path("mstatsdf", "full")) #hacky way of reading this file :)

In [2]:
# dirty_measurement df
dmdf = df[['driver_invocation', 'workload_invocation', 'provider', 'dirty_measurement', 'folder_uuid']]
dmdf

,driver_invocation,workload_invocation,provider,dirty_measurement,folder_uuid
0,2021-08-01 13:30:03.035,2021-08-01 13:30:42.746,AWS,True,81285f3e-341e-11ec-be3c-8bf2eba3159e
1,2021-08-01 13:30:03.035,2021-08-01 13:30:42.746,AWS,True,81285f3e-341e-11ec-be3c-8bf2eba3159e
2,2021-08-01 13:30:03.035,2021-08-01 13:30:42.746,AWS,True,81285f3e-341e-11ec-be3c-8bf2eba3159e
3,2021-08-01 13:30:03.035,2021-08-01 13:30:42.746,AWS,True,81285f3e-341e-11ec-be3c-8bf2eba3159e
4,2021-08-01 13:30:03.035,2021-08-01 13:30:42.746,AWS,True,81285f3e-341e-11ec-be3c-8bf2eba3159e
...,...,...,...,...,...
49935595,2021-09-30 23:30:03.438,2021-09-30 23:30:22.629,GCP,True,c0093932-341b-11ec-be3c-8bf2eba3159e
49935596,2021-09-30 23:30:03.438,2021-09-30 23:30:22.629,GCP,True,c0093932-341b-11ec-be3c-8bf2eba3159e
49935597,2021-09-30 23:30:03.438,2021-09-30 23:30:22.629,GCP,True,c0093932-341b-11ec-be3c-8bf2eba3159e
49935598,2021-09-30 23:30:03.438,2021-09-30 23:30:22.629,GCP,True,c0093932-341b-11ec-be3c-8bf2eba3159e


In [3]:
dmdf[['provider', 'dirty_measurement', 'driver_invocation']].groupby(['provider','dirty_measurement']).count()

driver_invocation
provider dirty_measurement                   
AWS      False                         995399
         True                        16361278
AZURE    False                         824143
         True                        14354841
GCP      False                        1041552
         True                        16358387

In [4]:
mdf
# count all region occurences where value = 0 --> measurement missing
# count alle region occurecnes where value = 1 --> measurement ok
# count all regions occurences where value != 1 --> measurement 'dirty'
# count all region occurneces where value > 1 --> measured too often

,azure_brazilsouth,gcp_southamerica-east1,aws_sa-east-1,aws_ca-central-1,gcp_northamerica-northeast1,aws_us-east-1,azure_eastus,gcp_us-east4,aws_us-west-1,aws_us-west-2,...,gcp_europe-west3,aws_ap-south-1,gcp_asia-south1,azure_centralindia,aws_ap-northeast-1,gcp_asia-northeast1,azure_japaneast,aws_ap-southeast-2,azure_australiaeast,gcp_australia-southeast1
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3049,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3050,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3051,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3052,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [5]:
dirty_measurement_df = mdf[(mdf == 0).any(axis=1)]
dirty_measurement_df

,azure_brazilsouth,gcp_southamerica-east1,aws_sa-east-1,aws_ca-central-1,gcp_northamerica-northeast1,aws_us-east-1,azure_eastus,gcp_us-east4,aws_us-west-1,aws_us-west-2,...,gcp_europe-west3,aws_ap-south-1,gcp_asia-south1,azure_centralindia,aws_ap-northeast-1,gcp_asia-northeast1,azure_japaneast,aws_ap-southeast-2,azure_australiaeast,gcp_australia-southeast1
42,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
54,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
58,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
84,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1
93,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2932,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2948,1,1,1,1,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2991,1,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [6]:
dirty_measurement_df

,azure_brazilsouth,gcp_southamerica-east1,aws_sa-east-1,aws_ca-central-1,gcp_northamerica-northeast1,aws_us-east-1,azure_eastus,gcp_us-east4,aws_us-west-1,aws_us-west-2,...,gcp_europe-west3,aws_ap-south-1,gcp_asia-south1,azure_centralindia,aws_ap-northeast-1,gcp_asia-northeast1,azure_japaneast,aws_ap-southeast-2,azure_australiaeast,gcp_australia-southeast1
42,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
54,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
58,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
84,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1
93,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2932,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2948,1,1,1,1,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2991,1,1,1,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
